## Setup: Load Example Data

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
from dosemetrics import read_dose_and_mask_files

# Download dataset
data_path = Path(snapshot_download(
    repo_id="contouraid/dosemetrics-data",
    repo_type="dataset"
))

# Load test subject
subject_path = data_path / "longitudinal" / "time_point_1"
dose, structures = read_dose_and_mask_files(subject_path)

print(f"✓ Loaded dose: {dose.shape}")
print(f"✓ Loaded {len(structures.structure_names)} structures")

## 1. Computing Dose-Volume Histograms (DVH)

The DVH is the fundamental metric in radiotherapy planning. It shows what percentage of a structure receives a given dose.

In [ ]:
from dosemetrics import compute_dvh

# Compute DVH for target
ptv_mask = structures.get_structure_mask("PTV")
dvh = compute_dvh(dose, ptv_mask, organ_name="PTV")

print("PTV DVH computed:")
print(f"  Number of bins: {len(dvh.dose_bins)}")
print(f"  Dose range: {dvh.dose_bins[0]:.2f} - {dvh.dose_bins[-1]:.2f} Gy")
print(f"  Volume fractions: {dvh.volume_fractions.min():.3f} - {dvh.volume_fractions.max():.3f}")

## 2. Extracting Dosimetric Parameters

Common dosimetric parameters:
- **Dx**: Dose received by x% of the volume (e.g., D95 = dose to 95% of volume)
- **Vx**: Volume receiving at least x Gy (e.g., V20 = volume receiving ≥20 Gy)
- **Mean dose**: Average dose across the structure
- **Max dose**: Maximum point dose

In [ ]:
# Extract key metrics from DVH
print("PTV Dose Metrics:")
print(f"  Mean dose: {dvh.mean_dose:.2f} Gy")
print(f"  Max dose: {dvh.max_dose:.2f} Gy")
print(f"  Min dose: {dvh.min_dose:.2f} Gy")
print(f"\nDose coverage:")
print(f"  D98 (near-minimum): {dvh.get_dose_at_volume(98):.2f} Gy")
print(f"  D95 (standard): {dvh.get_dose_at_volume(95):.2f} Gy")
print(f"  D50 (median): {dvh.get_dose_at_volume(50):.2f} Gy")
print(f"  D2 (near-maximum): {dvh.get_dose_at_volume(2):.2f} Gy")

# Volume receiving specific doses
print(f"\nVolume coverage:")
print(f"  V95% (vol receiving ≥95% of prescription): {dvh.get_volume_at_dose(dvh.max_dose * 0.95):.1f}%")
print(f"  V107% (vol receiving ≥107% of prescription): {dvh.get_volume_at_dose(dvh.max_dose * 1.07):.1f}%")

## 3. Analyzing Organs at Risk (OARs)

For OARs, we typically care about maximum doses and volumes receiving high doses.

In [ ]:
# Analyze critical organs
oars = ["BrainStem", "Chiasm", "Eye_L", "Eye_R", "OpticNerve_L", "OpticNerve_R"]

print("Organs at Risk Analysis:")
print(f"{'Structure':20s} {'Mean (Gy)':>10s} {'Max (Gy)':>10s} {'D1cc (Gy)':>11s}")
print("-" * 55)

for oar_name in oars:
    if structures.has_structure(oar_name):
        mask = structures.get_structure_mask(oar_name)
        dvh_oar = compute_dvh(dose, mask, organ_name=oar_name)
        
        # D1cc - dose to 1 cubic centimeter (approximated)
        # For small structures, use D2% as proxy
        d1cc = dvh_oar.get_dose_at_volume(2)
        
        print(f"{oar_name:20s} {dvh_oar.mean_dose:10.2f} {dvh_oar.max_dose:10.2f} {d1cc:11.2f}")

## 4. Visualizing DVHs

Plot DVHs for multiple structures to visualize dose distributions.

In [ ]:
import matplotlib.pyplot as plt

# Structures to plot
structures_to_plot = [
    ("PTV", "red", "-"),
    ("Brain", "blue", "--"),
    ("BrainStem", "orange", "-"),
    ("Chiasm", "green", "-"),
    ("Eye_L", "purple", ":"),
    ("Eye_R", "purple", "-.")
]

fig, ax = plt.subplots(figsize=(12, 7))

for structure_name, color, linestyle in structures_to_plot:
    if structures.has_structure(structure_name):
        mask = structures.get_structure_mask(structure_name)
        dvh = compute_dvh(dose, mask, organ_name=structure_name)
        
        ax.plot(
            dvh.dose_bins,
            dvh.volume_fractions * 100,
            label=structure_name,
            color=color,
            linestyle=linestyle,
            linewidth=2
        )

ax.set_xlabel("Dose (Gy)", fontsize=12)
ax.set_ylabel("Volume (%)", fontsize=12)
ax.set_title("Dose-Volume Histogram", fontsize=14, fontweight='bold')
ax.legend(loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, None)
ax.set_ylim(0, 105)

plt.tight_layout()
plt.show()

## 5. Checking Dose Constraints

Evaluate whether a treatment plan meets clinical dose constraints.

In [ ]:
# Define clinical constraints (example values)
constraints = {
    "PTV": {
        "D95_min": 50.0,  # At least 50 Gy to 95% of target
        "D2_max": 70.0,   # No more than 70 Gy to hottest 2%
    },
    "BrainStem": {
        "max": 54.0,      # Max dose ≤ 54 Gy
    },
    "Chiasm": {
        "max": 54.0,      # Max dose ≤ 54 Gy
    },
    "Eye_L": {
        "max": 45.0,      # Max dose ≤ 45 Gy
    },
    "Eye_R": {
        "max": 45.0,      # Max dose ≤ 45 Gy
    },
}

# Check compliance
print("Dose Constraint Compliance:")
print(f"{'Structure':20s} {'Constraint':25s} {'Value':>10s} {'Limit':>10s} {'Status':>10s}")
print("="* 80)

all_pass = True
for structure_name, struct_constraints in constraints.items():
    if not structures.has_structure(structure_name):
        continue
    
    mask = structures.get_structure_mask(structure_name)
    dvh = compute_dvh(dose, mask, organ_name=structure_name)
    
    for constraint_name, limit in struct_constraints.items():
        if constraint_name == "max":
            value = dvh.max_dose
            passed = value <= limit
            constraint_str = "Max dose"
        elif constraint_name == "D95_min":
            value = dvh.get_dose_at_volume(95)
            passed = value >= limit
            constraint_str = "D95 ≥"
        elif constraint_name == "D2_max":
            value = dvh.get_dose_at_volume(2)
            passed = value <= limit
            constraint_str = "D2 ≤"
        else:
            continue
        
        status = "✓ PASS" if passed else "✗ FAIL"
        all_pass = all_pass and passed
        
        print(f"{structure_name:20s} {constraint_str:25s} {value:10.2f} {limit:10.2f} {status:>10s}")

print("="* 80)
print(f"Overall: {'✓ ALL CONSTRAINTS MET' if all_pass else '✗ SOME CONSTRAINTS VIOLATED'}")

## 6. Comparing Predicted vs Actual Dose

Evaluate the accuracy of machine learning dose predictions.

In [ ]:
from dosemetrics import read_from_nifti

# Load predicted dose
predicted_dose = read_from_nifti(str(subject_path / "Predicted_Dose.nii.gz"))

# Compare DVHs for Target
ptv_mask = structures.get_structure_mask("PTV")
dvh_actual = compute_dvh(dose, ptv_mask, organ_name="PTV (Actual)")
dvh_predicted = compute_dvh(predicted_dose, ptv_mask, organ_name="PTV (Predicted)")

print("Predicted vs Actual Dose Comparison (Target):")
print(f"{'Metric':20s} {'Actual':>12s} {'Predicted':>12s} {'Difference':>12s}")
print("-" * 60)

metrics = [
    ("Mean dose (Gy)", dvh_actual.mean_dose, dvh_predicted.mean_dose),
    ("Max dose (Gy)", dvh_actual.max_dose, dvh_predicted.max_dose),
    ("D95 (Gy)", dvh_actual.get_dose_at_volume(95), dvh_predicted.get_dose_at_volume(95)),
    ("D50 (Gy)", dvh_actual.get_dose_at_volume(50), dvh_predicted.get_dose_at_volume(50)),
]

for metric_name, actual_val, pred_val in metrics:
    diff = pred_val - actual_val
    print(f"{metric_name:20s} {actual_val:12.2f} {pred_val:12.2f} {diff:+12.2f}")

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 7))
ax.plot(dvh_actual.dose_bins, dvh_actual.volume_fractions * 100, 
        label='Actual', color='blue', linewidth=2.5)
ax.plot(dvh_predicted.dose_bins, dvh_predicted.volume_fractions * 100, 
        label='Predicted', color='red', linewidth=2.5, linestyle='--')
ax.set_xlabel("Dose (Gy)", fontsize=12)
ax.set_ylabel("Volume (%)", fontsize=12)
ax.set_title("Predicted vs Actual DVH (Target)", fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Comparing Treatment Plans

Compare dosimetric metrics between two different treatment plans.

In [ ]:
# Load two plans
plan1_path = data_path / "longitudinal" / "time_point_1"
plan2_path = data_path / "longitudinal" / "time_point_2"

dose1, structures1 = read_dose_and_mask_files(plan1_path)
dose2, structures2 = read_dose_and_mask_files(plan2_path)

# Compare for key structures
compare_structures = ["Brainstem", "Chiasm", "Eye_L", "Eye_R"]

print("Plan Comparison:")
print(f"{'Structure':15s} {'Metric':10s} {'Plan 1':>10s} {'Plan 2':>10s} {'Difference':>12s}")
print("="* 65)

for struct_name in compare_structures:
    if structures1.has_structure(struct_name) and structures2.has_structure(struct_name):
        mask1 = structures1.get_structure_mask(struct_name)
        mask2 = structures2.get_structure_mask(struct_name)
        
        dvh1 = compute_dvh(dose1, mask1, organ_name=struct_name)
        dvh2 = compute_dvh(dose2, mask2, organ_name=struct_name)
        
        # Compare mean dose
        diff_mean = dvh2.mean_dose - dvh1.mean_dose
        print(f"{struct_name:15s} {'Mean':10s} {dvh1.mean_dose:10.2f} {dvh2.mean_dose:10.2f} {diff_mean:+12.2f}")
        
        # Compare max dose
        diff_max = dvh2.max_dose - dvh1.max_dose
        print(f"{' ':15s} {'Max':10s} {dvh1.max_dose:10.2f} {dvh2.max_dose:10.2f} {diff_max:+12.2f}")
        print()

## 8. Computing Quality Indices

Quality indices provide a single metric to assess plan quality.

In [ ]:
# Compute conformity and homogeneity indices for target
ptv_mask = structures.get_structure_mask("PTV")
dvh_target = compute_dvh(dose, ptv_mask, organ_name="PTV")

# Homogeneity Index (HI) = (D2 - D98) / D50
# Lower is better (more homogeneous)
d2 = dvh_target.get_dose_at_volume(2)
d98 = dvh_target.get_dose_at_volume(98)
d50 = dvh_target.get_dose_at_volume(50)
hi = (d2 - d98) / d50 if d50 > 0 else 0

print("PTV Quality Indices:")
print(f"  Homogeneity Index (HI): {hi:.3f}")
print(f"    D2:  {d2:.2f} Gy")
print(f"    D98: {d98:.2f} Gy")
print(f"    D50: {d50:.2f} Gy")
print(f"\n  Interpretation:")
if hi < 0.15:
    print(f"    ✓ Excellent homogeneity (HI < 0.15)")
elif hi < 0.25:
    print(f"    ✓ Good homogeneity (0.15 ≤ HI < 0.25)")
else:
    print(f"    ! May need improvement (HI ≥ 0.25)")

# Coverage metrics
v95 = dvh_target.get_volume_at_dose(d50 * 0.95)
print(f"\n  Coverage:")
print(f"    V95%: {v95:.1f}% of target receives ≥95% of prescription")
if v95 >= 95:
    print(f"    ✓ Excellent coverage (V95% ≥ 95%)")
elif v95 >= 90:
    print(f"    ✓ Acceptable coverage (90% ≤ V95% < 95%)")
else:
    print(f"    ! May need improvement (V95% < 90%)")

## Summary

In this notebook, you learned how to:

1. ✓ Compute Dose-Volume Histograms (DVH)
2. ✓ Extract dosimetric parameters (Dx, Vx)
3. ✓ Analyze organs at risk
4. ✓ Visualize DVHs
5. ✓ Check dose constraint compliance
6. ✓ Compare predicted vs actual doses
7. ✓ Compare treatment plans
8. ✓ Compute quality indices

## Next Steps

- **Exporting Results**: Generate reports, CSV files, and publication-ready plots
- **Batch Processing**: Analyze multiple patients at once
- **Advanced Analysis**: Gamma analysis, geometric comparisons, statistical testing

## References

- [DoseMetrics Documentation](https://contouraid.github.io/dosemetrics/)
- [DVH Analysis Guide](https://contouraid.github.io/dosemetrics/user-guide/dvh-analysis/)
- [GitHub Repository](https://github.com/contouraid/dosemetrics)